# Mileage prediction

In [1]:
import numpy as np
import pandas as pd
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 50

register_data_path = '../../vehicle_register/data/'
inspections_data_path = '../../inspections/data/'


In [2]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())

I see 1 GPU devices


## Load data

### Vehicle register

In [2]:
register_cols = [
    'VIN',
    'Type',
    'SecondaryType',
    'Category',
    'Make',
    'Model',
    'ManufactureYear',
    # 'FirstRegistration',
    'MotorPower',
    'MotorVolume',
    'DriveType',
    'Places',
    'Length',
    'Width',
    'Height',
    'OperatingWeight',
    'PermissibleWeight',
    'MaxSpeed',
    'AverageConsumption',
    'CityConsumption',
    'OutOfCityConsumption',
    'Emissions',
    'CityEmissions',
    'OutOfCityEmissions',
]

register_dtypes = {
    'VIN': 'str',
    'Type': 'category',
    'SecondaryType': 'category',
    'Category': 'category',
    'Make': 'category',
    'Model': 'category',
    'ManufactureYear': 'float16',
    # 'FirstRegistration': '',
    'MotorPower': 'float16',
    'MotorVolume': 'float16',
    'DriveType': 'category',
    'Places': 'float16',
    'Length': 'float16',
    'Width': 'float16',
    'Height': 'float16',
    'OperatingWeight': 'float16',
    'PermissibleWeight': 'float16',
    'MaxSpeed': 'float16',
    'AverageConsumption': 'float16',
    'CityConsumption': 'float16',
    'OutOfCityConsumption': 'float16',
    'Emissions': 'float16',
    'CityEmissions': 'float16',
    'OutOfCityEmissions': 'float16',
}

df_register = pd.read_csv(
    f'{register_data_path}nosync/registr_silnicnich_vozidel_2023-02-24.analysis-cleaned-2023-05-14.csv',
    usecols=register_cols,
    dtype=register_dtypes,
    # parse_dates=['FirstRegistration'],
    true_values=['True'],
    false_values=['False'],
    encoding_errors='ignore')

In [5]:
df_register.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4334574 entries, 0 to 4334573
Data columns (total 23 columns):
 #   Column                Dtype   
---  ------                -----   
 0   ManufactureYear       float16 
 1   Type                  category
 2   SecondaryType         category
 3   Category              category
 4   Make                  category
 5   Model                 category
 6   VIN                   object  
 7   MotorPower            float16 
 8   MotorVolume           float16 
 9   DriveType             category
 10  Places                float16 
 11  Length                float16 
 12  Width                 float16 
 13  Height                float16 
 14  OperatingWeight       float16 
 15  PermissibleWeight     float16 
 16  MaxSpeed              float16 
 17  CityConsumption       float16 
 18  AverageConsumption    float16 
 19  OutOfCityConsumption  float16 
 20  Emissions             float16 
 21  CityEmissions         float16 
 22  OutOfCityEmissions

### Inspections

In [3]:
from genericpath import exists


inspections_cols = [
    # 'StationId', # Inspection result shouldn't depend on station.
    # 'Type',
    'VIN',
    'Date',
    # 'MotorType',
    # 'Make',
    # 'VehicleType',
    # 'Model',
    # 'VehicleClass',
    # 'FirstRegistrationDate',
    'Km',
    # 'Defects',
    'Result',
    # "Weekday",
    "DefectsA",
    "DefectsB",
    "DefectsC",
    "Defects0",
    "Defects1",
    "Defects2",
    "Defects3",
    "Defects4",
    "Defects5",
    "Defects6",
    "Defects7",
    "Defects8",
    "Defects9",
    "AgeDays"
]

dtypes = {
    'StationId': 'category',
    # 'Type': 'category',
    # 'Make': 'category',
    # 'Model': 'category',
    # 'MotorType': 'category',
    # 'VehicleClass': 'category',
    'Result': 'category',
    # "Weekday": 'int8',
    # "DefectsA": 'int8',
    # "DefectsB": 'int8',
    # "DefectsC": 'int8',
    # "Defects0": 'int8',
    # "Defects1": 'int8',
    # "Defects2": 'int8',
    # "Defects3": 'int8',
    # "Defects4": 'int8',
    # "Defects5": 'int8',
    # "Defects6": 'int8',
    # "Defects7": 'int8',
    # "Defects8": 'int8',
    # "Defects9": 'int8',
}


def load_year(year):
    df_all = None

    for name in [f'{year}-{month:02}' for month in range(1, 13)]:
        # print(f'Load: {name}')
        if exists(f'{inspections_data_path}nosync/data_csv/{name}.csv'):
            df = pd.read_csv(f'{inspections_data_path}nosync/data_csv/{name}.csv',
                             dtype=dtypes, usecols=inspections_cols, index_col=False)
            df['Date'] = pd.to_datetime(df['Date'])
            # df['FirstRegistrationDate'] = pd.to_datetime(
            #     df['FirstRegistrationDate'])

        if df_all is None:
            df_all = df
        else:
            df_all = pd.concat([df_all, df])

    return df_all


df_inspections = None

for year in [
    # '2018',
    '2019',
    '2020',
    '2021',
    '2022'
]:
    print(year)
    df = load_year(year)
    if df_inspections is None:
        df_inspections = df
    else:
        df_inspections = pd.concat([df_inspections, df])

df_inspections = df_inspections.drop_duplicates(keep='first')
# TODO: Investigate source of duplicates.


2019
2020
2021
2022


In [71]:
df_inspections.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7868631 entries, 0 to 175762
Data columns (total 18 columns):
 #   Column    Dtype         
---  ------    -----         
 0   VIN       object        
 1   Date      datetime64[ns]
 2   Km        int64         
 3   Result    category      
 4   DefectsA  int64         
 5   DefectsB  int64         
 6   DefectsC  int64         
 7   Defects0  int64         
 8   Defects1  int64         
 9   Defects2  int64         
 10  Defects3  int64         
 11  Defects4  int64         
 12  Defects5  int64         
 13  Defects6  int64         
 14  Defects7  int64         
 15  Defects8  int64         
 16  Defects9  int64         
 17  AgeDays   int64         
dtypes: category(1), datetime64[ns](1), int64(15), object(1)
memory usage: 1.1+ GB


In [72]:
df_inspections

,VIN,Date,Km,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays
0,TAH31100EF1112050,2019-01-16 11:36:42.417,18719,způsobilé,3,0,0,0,1,0,0,0,2,1,0,0,0,1105
1,TN9FV1Z00RAAM5032,2019-01-09 09:42:53.517,85753,způsobilé,3,0,0,0,1,0,0,1,0,1,0,0,0,9139
2,RLVMEM5057VN00070,2019-01-31 15:21:19.753,15380,částečně způsobilé,0,1,0,1,0,0,0,0,0,0,0,0,0,4447
3,ZAR162B4000075261,2019-01-14 07:37:22.723,239430,způsobilé,6,0,0,0,2,0,0,1,1,3,0,0,0,10970
4,ZAR93000002261774,2019-01-29 10:58:24.557,218070,způsobilé,3,0,0,0,2,0,0,0,1,1,0,0,0,6823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175758,MMBJNK7402D038238,2022-11-30 00:00:00.000,305721,způsobilé,7,0,0,0,0,1,2,1,1,2,0,0,0,7484
175759,SHSRD87602U028813,2022-11-30 00:00:00.000,226502,způsobilé,3,0,0,0,1,0,0,1,0,1,0,0,0,7230
175760,WVWZZZ1JZ2D350006,2022-11-30 00:00:00.000,384127,způsobilé,5,0,0,0,1,0,1,0,1,2,0,0,0,7595
175761,WF0GXXGBBGAL35307,2022-11-30 00:00:00.000,291165,způsobilé,4,0,0,0,0,0,0,1,1,2,0,0,0,4571


## Preprocess data

### Prepare features

In [4]:
df_inspections['Result'] = df_inspections['Result'].cat.set_categories(['nezpůsobilé', 'částečně způsobilé', 'způsobilé'], ordered=True)

In [5]:
def split_drive_type(row):
    drive_type = str(row['DriveType'])
    petrol = diesel = gas = electric = other = False
    
    if 'Benzin' in drive_type or 'BA' in drive_type:
        petrol = True
    if 'Nafta' in drive_type:
        diesel = True
    if 'LPG' in drive_type or 'CNG' in drive_type or 'LNG' in drive_type or 'Metan' in drive_type:
        gas = True
    if 'Elektro' in drive_type:
        electric = True
    if 'Etanol' in drive_type or 'Vodík' in drive_type:
        other = True

    return petrol, diesel, gas, electric, other

df_register[['DriveTypePetrol', 'DriveTypeDiesel', 'DriveTypeGas', 'DriveTypeElectric', 'DriveTypeOther']] = df_register.apply(split_drive_type, axis=1, result_type='expand')
df_register = df_register.drop('DriveType', axis=1)

In [26]:
df_register.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4334574 entries, 0 to 4334573
Data columns (total 27 columns):
 #   Column                Dtype   
---  ------                -----   
 0   ManufactureYear       float16 
 1   Type                  category
 2   SecondaryType         category
 3   Category              category
 4   Make                  category
 5   Model                 category
 6   VIN                   object  
 7   MotorPower            float16 
 8   MotorVolume           float16 
 9   Places                float16 
 10  Length                float16 
 11  Width                 float16 
 12  Height                float16 
 13  OperatingWeight       float16 
 14  PermissibleWeight     float16 
 15  MaxSpeed              float16 
 16  CityConsumption       float16 
 17  AverageConsumption    float16 
 18  OutOfCityConsumption  float16 
 19  Emissions             float16 
 20  CityEmissions         float16 
 21  OutOfCityEmissions    float16 
 22  DriveTypePetrol   

### Add initial state for each vehicle
Each vehicle has a mileage and defect counts equal to zero when new.
Add this state as an additional inspection data point. 

In [6]:
def create_initial_state() -> pd.DataFrame:
    vin = df_inspections['VIN'].unique()
    
    df_inspections_init = pd.DataFrame({'VIN': vin})
    df_inspections_init['Km'] = 0
    df_inspections_init['Result'] = 'způsobilé'
    for index in ['A', 'B', 'C']:
        df_inspections_init[f'Defects{index}'] = 0
    for index in range(0, 10):
        df_inspections_init[f'Defects{index}'] = 0
    df_inspections_init['AgeDays'] = 0

    df_inspections_init['Result'] = df_inspections_init['Result'].astype('category')
    df_inspections_init['Result'] = df_inspections_init['Result'].cat.set_categories(['nezpůsobilé', 'částečně způsobilé', 'způsobilé'], ordered=True)

    return df_inspections_init

# create_initial_state()
df_inspections = pd.concat([create_initial_state(), df_inspections], axis=0)
df_inspections.reset_index()
df_inspections

,VIN,Km,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,Date
0,TAH31100EF1112050,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
1,TN9FV1Z00RAAM5032,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
2,RLVMEM5057VN00070,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
3,ZAR162B4000075261,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
4,ZAR93000002261774,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175758,MMBJNK7402D038238,305721,způsobilé,7,0,0,0,0,1,2,1,1,2,0,0,0,7484,2022-11-30
175759,SHSRD87602U028813,226502,způsobilé,3,0,0,0,1,0,0,1,0,1,0,0,0,7230,2022-11-30
175760,WVWZZZ1JZ2D350006,384127,způsobilé,5,0,0,0,1,0,1,0,1,2,0,0,0,7595,2022-11-30
175761,WF0GXXGBBGAL35307,291165,způsobilé,4,0,0,0,0,0,0,1,1,2,0,0,0,4571,2022-11-30


### Handle NaNs in categorical values
Catboost doesn't accept NaNs in categorical columns. Replace them with a special category.

In [7]:
for category in [
        'Type',
        'SecondaryType',
        'Category',
        'Make',
        'Model',
        'DriveTypePetrol',
        'DriveTypeDiesel',
        'DriveTypeGas',
        'DriveTypeElectric',
        'DriveTypeOther']:
    if df_register[category].isna().sum() > 0:
        new_categories = df_register[category].cat.categories.to_list()
        new_categories.append('None')
        df_register[category] = df_register[category].cat.set_categories(new_categories)
        df_register[category] = df_register[category].fillna('None')

In [43]:
df_register[df_register['Model'] == 'None']

,ManufactureYear,Type,SecondaryType,Category,Make,Model,VIN,MotorPower,MotorVolume,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
1,1998.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,VW,None,WVWZZZ6XZXW008023,55.0,1390.0,4.0,3524.0,1639.0,1460.0,988.0,1380.0,172.0,8.296875,5.000000,6.199219,NaN,NaN,NaN,True,False,False,False,False
2,1991.0,OSOBNÍ AUTOMOBIL,None,M1,NISSAN,None,JN1BCAN14U0758870,55.0,1392.0,5.0,4232.0,1670.0,1395.0,985.0,1475.0,170.0,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False
3,1993.0,OSOBNÍ AUTOMOBIL,OBYTNÝ,M1,PEUGEOT,None,VF3290G5200371940,54.0,2500.0,7.0,5840.0,2200.0,2880.0,2444.0,3100.0,115.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
6,2003.0,OSOBNÍ AUTOMOBIL,OSOBNÍ,M1,DAEWOO,None,KLASF48T13B058479,53.0,1150.0,5.0,NaN,NaN,NaN,1035.0,1455.0,157.0,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False
10,1992.0,NÁKLADNÍ AUTOMOBIL,None,N1,SUZUKI,None,JSAOSJ70V00402850,51.0,1298.0,2.0,3440.0,1540.0,1675.0,1025.0,1330.0,130.0,9.101562,7.500000,13.296875,NaN,NaN,NaN,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334567,2014.0,OSOBNÍ AUTOMOBIL,VÍCEÚČELOVÉ VOZIDLO,M1,OPEL,None,W0L2J7200FV611199,88.0,1598.0,9.0,5000.0,1956.0,1951.0,1976.0,2800.0,172.0,6.398438,5.300781,5.699219,149.0,166.0,139.0,False,True,False,False,False
4334568,2019.0,NÁKLADNÍ AUTOMOBIL,PICK-UP,N1,FORD,None,6FPPXXMJ2PKE00139,147.0,3198.0,5.0,5360.0,1867.0,1815.0,2344.0,3270.0,175.0,10.296875,7.800781,8.796875,225.0,265.0,202.0,False,True,False,False,False
4334570,2022.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,SSANGYONG,None,KPT60A1YSNP081280,120.0,1497.0,5.0,4448.0,1870.0,1620.0,1517.0,1980.0,191.0,NaN,NaN,7.601562,175.0,NaN,NaN,True,False,False,False,False
4334572,2017.0,OSOBNÍ AUTOMOBIL,SEDAN,M1,SUBARU,None,JF1VAFLH3JG013329,221.0,2456.0,5.0,4596.0,1795.0,1475.0,1597.0,2000.0,255.0,15.398438,8.703125,11.203125,259.0,361.0,200.0,True,False,False,False,False


In [61]:
df_register[df_register['VIN'] == 'ZAR93200001106590']

,ManufactureYear,Type,SecondaryType,Category,Make,Model,VIN,MotorPower,MotorVolume,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
3014930,1999.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,ZAR93200001106590,77.0,1910.0,5.0,4432.0,1745.0,1415.0,1345.0,1770.0,186.0,7.800781,4.699219,5.800781,NaN,NaN,NaN,False,True,False,False,False


In [75]:
df_inspections[df_inspections['VIN'] == 'ZAR93200001106590']

,VIN,Km,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,Date
142,ZAR93200001106590,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
148,ZAR93200001106590,150610,způsobilé,3,0,0,0,0,1,0,0,1,2,0,0,0,7158,2019-01-21 14:45:15.337
75896,ZAR93200001106590,161154,způsobilé,4,0,0,0,0,1,0,1,0,2,0,0,0,7852,2020-12-15 00:00:00.000
144232,ZAR93200001106590,170489,způsobilé,4,0,0,0,0,1,1,0,0,2,0,0,0,8561,2022-11-24 00:00:00.000


### Join datasets

In [8]:
df_full = df_inspections.merge(df_register, on='VIN', how='inner')
# df_register = None
# df_inspections = None

In [77]:
df_full.iloc[178:181]

,VIN,Km,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,Date,ManufactureYear,Type,SecondaryType,Category,Make,Model,MotorPower,MotorVolume,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
178,ZAR93200001106590,161154,způsobilé,4,0,0,0,0,1,0,1,0,2,0,0,0,7852,2020-12-15,1999.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4432.0,1745.0,1415.0,1345.0,1770.0,186.0,7.800781,4.699219,5.800781,NaN,NaN,NaN,False,True,False,False,False
179,ZAR93200001106590,170489,způsobilé,4,0,0,0,0,1,1,0,0,2,0,0,0,8561,2022-11-24,1999.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4432.0,1745.0,1415.0,1345.0,1770.0,186.0,7.800781,4.699219,5.800781,NaN,NaN,NaN,False,True,False,False,False
180,ZAR93200001204406,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT,2000.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ALFA ROMEO,None,100.0,2388.0,5.0,4432.0,1745.0,1390.0,1475.0,1895.0,203.0,9.000000,5.601562,6.800781,NaN,NaN,NaN,False,True,False,False,False


In [14]:
# Verify result
# Find a VIN that appears multiple times in the inspections.
display(df_inspections['VIN'].value_counts())
display(df_full[df_full['VIN'] == 'WMAL33ZZZ3Y107270'])

WDB9320031K905830    15
WMAL33ZZZ3Y107270    14
VF647BCA000000575    13
XLRAE45GF0L367085    13
WMAH56ZZ97M480022    13
                     ..
TMBGY26Y454355348     2
VG5SG252000003196     2
WV2ZZZ7HZDH106315     2
WAUZZZ4AZVN028217     2
W0L0TGF48Y5087158     2
Name: VIN, Length: 4470091, dtype: int64

,VIN,Km,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,ManufactureYear,Type,SecondaryType,Category,Make,Model,MotorPower,MotorVolume,DriveType,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
69581,WMAL33ZZZ3Y107270,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69582,WMAL33ZZZ3Y107270,620359,částečně způsobilé,7,4,1,0,3,0,0,1,2,6,0,0,0,5903,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69583,WMAL33ZZZ3Y107270,622030,částečně způsobilé,7,1,0,0,2,0,0,0,1,5,0,0,0,5935,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69584,WMAL33ZZZ3Y107270,622019,nezpůsobilé,7,1,2,0,3,0,0,0,1,6,0,0,0,5932,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69585,WMAL33ZZZ3Y107270,623064,způsobilé,7,0,0,0,1,0,0,0,1,5,0,0,0,5963,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69586,WMAL33ZZZ3Y107270,642234,částečně způsobilé,3,3,0,0,1,0,0,3,0,2,1,0,0,6267,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69587,WMAL33ZZZ3Y107270,643161,způsobilé,3,0,0,0,1,0,0,1,0,2,0,0,0,6280,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69588,WMAL33ZZZ3Y107270,660629,částečně způsobilé,6,4,0,0,3,1,0,0,1,4,1,0,0,6651,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69589,WMAL33ZZZ3Y107270,661868,nezpůsobilé,6,2,0,0,3,1,0,0,1,3,0,0,0,6679,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
69590,WMAL33ZZZ3Y107270,661880,způsobilé,6,0,0,0,1,1,0,0,1,3,0,0,0,6680,2002.0,NÁKLADNÍ AUTOMOBIL,BA,N2,MAN,NaN,132.0,4580.0,Nafta,2.0,10384.0,2490.0,2550.0,5112.0,8600.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False


### Filter features

In [83]:
df_full.columns

Index(['VIN', 'Km', 'Result', 'DefectsA', 'DefectsB', 'DefectsC', 'Defects0',
       'Defects1', 'Defects2', 'Defects3', 'Defects4', 'Defects5', 'Defects6',
       'Defects7', 'Defects8', 'Defects9', 'AgeDays', 'Date',
       'ManufactureYear', 'Type', 'SecondaryType', 'Category', 'Make', 'Model',
       'MotorPower', 'MotorVolume', 'Places', 'Length', 'Width', 'Height',
       'OperatingWeight', 'PermissibleWeight', 'MaxSpeed', 'CityConsumption',
       'AverageConsumption', 'OutOfCityConsumption', 'Emissions',
       'CityEmissions', 'OutOfCityEmissions', 'DriveTypePetrol',
       'DriveTypeDiesel', 'DriveTypeGas', 'DriveTypeElectric',
       'DriveTypeOther'],
      dtype='object')

In [9]:
df_full = df_full[['VIN', 'Km', 'Result', 'DefectsA', 'DefectsB',
       'Defects1', 'Defects4', 'Defects5', 'Defects6',
       'AgeDays', 'Date',
       'ManufactureYear', 'Type', 'Make',
       'MotorPower', 'MotorVolume', 'Places',
       'OperatingWeight', 'PermissibleWeight', 'MaxSpeed', 'CityConsumption',
       'AverageConsumption', 'OutOfCityConsumption', 'DriveTypePetrol',
       'DriveTypeDiesel', 'DriveTypeGas', 'DriveTypeElectric',
       'DriveTypeOther']]

### Split datasets for training and evaluation

In [47]:
# X = df_full.drop(['VIN', 'Km'], axis=1)
# y = df_full['Km']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=420)

In [10]:
# Take inspections from the last 6 months as test set, resulting in roughly
# 7:1 train:test ratio.

df_train = df_full[~(df_full['Date'] >= '2022-07-01')]
df_val = df_full[(df_full['Date'] >= '2022-07-01') & (df_full['Date'] < '2022-10-01')]
df_test = df_full[df_full['Date'] >= '2022-10-01']

X_train = df_train.drop(['VIN', 'Date', 'Km'], axis=1)
y_train = df_train['Km']

X_val = df_val.drop(['VIN', 'Date', 'Km'], axis=1)
y_val = df_val['Km']

X_test = df_test.drop(['VIN', 'Date', 'Km'], axis=1)
y_test = df_test['Km']

In [53]:
X_train

,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,ManufactureYear,Type,SecondaryType,Category,Make,Model,MotorPower,MotorVolume,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2000.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4092.0,1712.0,1427.0,1285.0,1730.0,185.0,7.601562,4.699219,5.699219,NaN,NaN,NaN,False,True,False,False,False
1,způsobilé,4,0,0,0,2,0,1,0,1,1,0,0,0,6880,2000.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4092.0,1712.0,1427.0,1285.0,1730.0,185.0,7.601562,4.699219,5.699219,NaN,NaN,NaN,False,True,False,False,False
2,částečně způsobilé,3,3,0,0,0,0,1,0,2,3,0,0,0,7614,2000.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4092.0,1712.0,1427.0,1285.0,1730.0,185.0,7.601562,4.699219,5.699219,NaN,NaN,NaN,False,True,False,False,False
3,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1996.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,76.0,1596.0,5.0,4092.0,1712.0,1427.0,1215.0,1670.0,185.0,10.000000,6.101562,8.203125,NaN,NaN,NaN,True,False,False,False,False
4,způsobilé,6,0,0,0,4,0,0,0,0,2,0,0,0,8240,1996.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,76.0,1596.0,5.0,4092.0,1712.0,1427.0,1215.0,1670.0,185.0,10.000000,6.101562,8.203125,NaN,NaN,NaN,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7019316,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ŠKODA,SUPERB,103.0,1968.0,5.0,4840.0,1817.0,1529.0,1569.0,2132.0,205.0,6.898438,4.699219,5.500000,145.0,180.0,123.0,False,True,False,False,False
7019319,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,KIA,CEED,66.0,1396.0,5.0,4488.0,1790.0,1525.0,1317.0,1820.0,180.0,7.398438,5.398438,6.101562,NaN,NaN,NaN,True,False,False,False,False
7019322,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2004.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,PEUGEOT,206,55.0,1360.0,5.0,4028.0,1664.0,1443.0,1074.0,1499.0,174.0,8.500000,5.199219,6.398438,153.0,203.0,125.0,True,False,False,False,False
7019325,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ŠKODA,OCTAVIA,77.0,1896.0,5.0,4572.0,1769.0,1522.0,1400.0,1985.0,191.0,6.500000,4.398438,5.101562,NaN,NaN,NaN,False,True,False,False,False


In [54]:
X_test

,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,ManufactureYear,Type,SecondaryType,Category,Make,Model,MotorPower,MotorVolume,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
36,způsobilé,8,0,0,0,2,0,0,2,1,3,0,0,0,7173,2003.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,110.0,1970.0,5.0,4168.0,1729.0,1442.0,1345.0,1790.0,208.0,12.101562,7.800781,8.898438,NaN,NaN,NaN,True,False,False,False,False
179,způsobilé,4,0,0,0,0,1,1,0,0,2,0,0,0,8561,1999.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4432.0,1745.0,1415.0,1345.0,1770.0,186.0,7.800781,4.699219,5.800781,NaN,NaN,NaN,False,True,False,False,False
180,způsobilé,4,0,0,0,0,1,1,0,0,2,0,0,0,8561,1999.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4432.0,1745.0,1415.0,1345.0,1770.0,186.0,7.800781,4.699219,5.800781,NaN,NaN,NaN,False,True,False,False,False
214,částečně způsobilé,4,2,0,0,3,0,0,0,1,2,0,0,0,5970,2006.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ALFA ROMEO,None,147.0,2388.0,5.0,4660.0,1830.0,1452.0,1755.0,2200.0,228.0,9.398438,5.601562,7.000000,NaN,NaN,NaN,False,True,False,False,False
215,způsobilé,4,0,0,0,2,0,0,0,0,2,0,0,0,5979,2006.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ALFA ROMEO,None,147.0,2388.0,5.0,4660.0,1830.0,1452.0,1755.0,2200.0,228.0,9.398438,5.601562,7.000000,NaN,NaN,NaN,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7019324,způsobilé,1,0,0,0,0,0,0,0,0,1,0,0,0,6588,2004.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,PEUGEOT,206,55.0,1360.0,5.0,4028.0,1664.0,1443.0,1074.0,1499.0,174.0,8.500000,5.199219,6.398438,153.0,203.0,125.0,True,False,False,False,False
7019326,způsobilé,1,0,0,0,0,0,0,0,0,1,0,0,0,5095,2008.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ŠKODA,OCTAVIA,77.0,1896.0,5.0,4572.0,1769.0,1522.0,1400.0,1985.0,191.0,6.500000,4.398438,5.101562,NaN,NaN,NaN,False,True,False,False,False
7019327,způsobilé,1,0,0,0,0,0,0,0,0,1,0,0,0,5095,2008.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ŠKODA,OCTAVIA,77.0,1896.0,5.0,4572.0,1769.0,1522.0,1400.0,1985.0,191.0,6.500000,4.398438,5.101562,NaN,NaN,NaN,False,True,False,False,False
7019329,způsobilé,4,0,0,0,0,0,0,1,1,2,0,0,0,4571,2010.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,FORD,MONDEO,103.0,1997.0,5.0,4856.0,1886.0,1548.0,1597.0,2300.0,200.0,9.703125,5.500000,7.101562,NaN,NaN,NaN,False,True,False,False,False


### Create CB datasets

In [11]:
cat_features = [
    'Result',
    'Type',
    # 'SecondaryType',
    # 'Category',
    'Make',
    # 'Model',
    'DriveTypePetrol',
    'DriveTypeDiesel',
    'DriveTypeGas',
    'DriveTypeElectric',
    'DriveTypeOther'
]

train_dataset = cb.Pool(data=X_train, label=y_train, cat_features=cat_features)
val_dataset = cb.Pool(data=X_val, label=y_val, cat_features=cat_features)
test_dataset = cb.Pool(data=X_test, label=y_test, cat_features=cat_features)

## Train

In [16]:
model = cb.CatBoostRegressor(
    iterations=200,
    loss_function='RMSE',
    depth=8,
    l2_leaf_reg=2,
    learning_rate=0.5,
    # use_best_model=True,
    # eval_metric='RMSE',
    # od_type='Iter',
    task_type='GPU',
)

In [ ]:
model.fit(
    train_dataset,
    eval_set=val_dataset,
    plot=True
)

Performance using time-based split of last 6 months:

- All features
```
    Without initial state (0 age and mileage):
    RMSE: 77895.73
    R2: 0.50

    With initial state:
    RMSE: 78224.4848
    R2: 0.4996

    With initial state, filtered duplicates:
    RMSE: 78191.5701
    R2: 0.4987
```
- Reduced features
```
    RMSE: 79600.9181
    R2: 0.4804 
```
Using train_test_split:

    RMSE: 61162.96
    R2: 0.79

## Evaluate

In [ ]:
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)

print('Testing performance')
print('RMSE: {:.4f}'.format(rmse))
print('R2: {:.4f}'.format(r2))

Testing performance
RMSE: 79496.6426
R2: 0.4826


In [ ]:
# using train_test_split
# RMSE: 61162.96
# R2: 0.79

In [57]:
X_test

,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,ManufactureYear,Type,SecondaryType,Category,Make,Model,MotorPower,MotorVolume,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
36,způsobilé,8,0,0,0,2,0,0,2,1,3,0,0,0,7173,2003.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,110.0,1970.0,5.0,4168.0,1729.0,1442.0,1345.0,1790.0,208.0,12.101562,7.800781,8.898438,NaN,NaN,NaN,True,False,False,False,False
179,způsobilé,4,0,0,0,0,1,1,0,0,2,0,0,0,8561,1999.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4432.0,1745.0,1415.0,1345.0,1770.0,186.0,7.800781,4.699219,5.800781,NaN,NaN,NaN,False,True,False,False,False
180,způsobilé,4,0,0,0,0,1,1,0,0,2,0,0,0,8561,1999.0,OSOBNÍ AUTOMOBIL,None,M1,ALFA ROMEO,None,77.0,1910.0,5.0,4432.0,1745.0,1415.0,1345.0,1770.0,186.0,7.800781,4.699219,5.800781,NaN,NaN,NaN,False,True,False,False,False
214,částečně způsobilé,4,2,0,0,3,0,0,0,1,2,0,0,0,5970,2006.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ALFA ROMEO,None,147.0,2388.0,5.0,4660.0,1830.0,1452.0,1755.0,2200.0,228.0,9.398438,5.601562,7.000000,NaN,NaN,NaN,False,True,False,False,False
215,způsobilé,4,0,0,0,2,0,0,0,0,2,0,0,0,5979,2006.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ALFA ROMEO,None,147.0,2388.0,5.0,4660.0,1830.0,1452.0,1755.0,2200.0,228.0,9.398438,5.601562,7.000000,NaN,NaN,NaN,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7019324,způsobilé,1,0,0,0,0,0,0,0,0,1,0,0,0,6588,2004.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,PEUGEOT,206,55.0,1360.0,5.0,4028.0,1664.0,1443.0,1074.0,1499.0,174.0,8.500000,5.199219,6.398438,153.0,203.0,125.0,True,False,False,False,False
7019326,způsobilé,1,0,0,0,0,0,0,0,0,1,0,0,0,5095,2008.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ŠKODA,OCTAVIA,77.0,1896.0,5.0,4572.0,1769.0,1522.0,1400.0,1985.0,191.0,6.500000,4.398438,5.101562,NaN,NaN,NaN,False,True,False,False,False
7019327,způsobilé,1,0,0,0,0,0,0,0,0,1,0,0,0,5095,2008.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,ŠKODA,OCTAVIA,77.0,1896.0,5.0,4572.0,1769.0,1522.0,1400.0,1985.0,191.0,6.500000,4.398438,5.101562,NaN,NaN,NaN,False,True,False,False,False
7019329,způsobilé,4,0,0,0,0,0,0,1,1,2,0,0,0,4571,2010.0,OSOBNÍ AUTOMOBIL,KOMBI,M1,FORD,MONDEO,103.0,1997.0,5.0,4856.0,1886.0,1548.0,1597.0,2300.0,200.0,9.703125,5.500000,7.101562,NaN,NaN,NaN,False,True,False,False,False


In [ ]:
pred

array([218195.71980929, 268950.57014537, 268950.57014537, ...,
       248020.19366187, 265075.60257244, 265075.60257244])

In [ ]:
y_test

36         156585
179        170489
180        170489
214        243217
215        243222
            ...  
7019324    108580
7019326    162969
7019327    162969
7019329    291165
7019330    291165
Name: Km, Length: 501045, dtype: int64

In [ ]:
# Crop negative values
pred_crop = np.clip(a=pred, a_min=0, a_max=np.inf)
rmse = (np.sqrt(mean_squared_error(y_test, pred_crop)))
r2 = r2_score(y_test, pred_crop)

print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

Testing performance
RMSE: 77895.43
R2: 0.50


## Parameter tuning

In [15]:
grid = {'iterations': [200],
        'learning_rate': [0.25, 0.5],
        'depth': [6, 8],
        'l2_leaf_reg': [2, 3]}

model = cb.CatBoostRegressor(loss_function='RMSE', task_type='GPU')
model.grid_search(grid, train_dataset, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 147572.6866581	test: 147424.8262413	best: 147424.8262413 (0)	total: 216ms	remaining: 43s
1:	learn: 119387.6447140	test: 119265.0896373	best: 119265.0896373 (1)	total: 425ms	remaining: 42.1s
2:	learn: 99941.2153176	test: 99844.1776313	best: 99844.1776313 (2)	total: 645ms	remaining: 42.3s
3:	learn: 87050.2767742	test: 86978.4559091	best: 86978.4559091 (3)	total: 857ms	remaining: 42s
4:	learn: 78713.6161316	test: 78661.6502055	best: 78661.6502055 (4)	total: 1.07s	remaining: 41.7s
5:	learn: 73530.0139974	test: 73496.8589426	best: 73496.8589426 (5)	total: 1.28s	remaining: 41.4s
6:	learn: 70151.0117194	test: 70134.1689087	best: 70134.1689087 (6)	total: 1.49s	remaining: 41s
7:	learn: 68007.6720141	test: 68008.6863813	best: 68008.6863813 (7)	total: 1.69s	remaining: 40.5s
8:	learn: 66649.9514728	test: 66663.2171829	best: 66663.2171829 (8)	total: 1.89s	remaining: 40.1s
9:	learn: 65836.0546596	test: 65858.2274745	best: 65858.2274745 (9)	total: 2.09s	remaining: 39.8s
10:	learn: 65277.734

{'params': {'depth': 8,
  'l2_leaf_reg': 2,
  'iterations': 200,
  'learning_rate': 0.5},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

### Example prediction

In [ ]:
# Find a vehicle with more observations
vc = df_full['VIN'].value_counts()
vc[vc == 4]

In [30]:
test_observation = df_full[df_full['VIN'] == 'TMBPW16Y233650368']

display(test_observation)


,VIN,Km,Result,DefectsA,DefectsB,DefectsC,Defects0,Defects1,Defects2,Defects3,Defects4,Defects5,Defects6,Defects7,Defects8,Defects9,AgeDays,Date,ManufactureYear,Type,SecondaryType,Category,Make,Model,MotorPower,MotorVolume,Places,Length,Width,Height,OperatingWeight,PermissibleWeight,MaxSpeed,CityConsumption,AverageConsumption,OutOfCityConsumption,Emissions,CityEmissions,OutOfCityEmissions,DriveTypePetrol,DriveTypeDiesel,DriveTypeGas,DriveTypeElectric,DriveTypeOther
5914619,TMBPW16Y233650368,0,způsobilé,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT,2002.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,ŠKODA,FABIA,40.0,1198.0,5.0,3960.0,1646.0,1451.0,1130.0,1580.0,151.0,7.800781,4.800781,5.898438,NaN,NaN,NaN,True,False,False,False,False
5914620,TMBPW16Y233650368,234661,způsobilé,4,0,0,0,0,0,0,1,0,3,0,0,0,6572,2020-10-20,2002.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,ŠKODA,FABIA,40.0,1198.0,5.0,3960.0,1646.0,1451.0,1130.0,1580.0,151.0,7.800781,4.800781,5.898438,NaN,NaN,NaN,True,False,False,False,False
5914621,TMBPW16Y233650368,255037,způsobilé,4,0,0,1,0,0,1,0,0,2,0,0,0,7323,2022-11-10,2002.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,ŠKODA,FABIA,40.0,1198.0,5.0,3960.0,1646.0,1451.0,1130.0,1580.0,151.0,7.800781,4.800781,5.898438,NaN,NaN,NaN,True,False,False,False,False
5914622,TMBPW16Y233650368,255037,způsobilé,4,0,0,1,0,0,1,0,0,2,0,0,0,7323,2022-11-10,2002.0,OSOBNÍ AUTOMOBIL,HATCHBACK,M1,ŠKODA,FABIA,40.0,1198.0,5.0,3960.0,1646.0,1451.0,1130.0,1580.0,151.0,7.800781,4.800781,5.898438,NaN,NaN,NaN,True,False,False,False,False


In [42]:
# Predict the last observation
model.predict(X_test.loc[5914619])
# 5914622

KeyError: 5914619

In [ ]:
# Construct a few future observations
